# Digital Finger Printing (DFP) with Morpheus - DUO Inference
## Introduction

In this notebook, we will be building and running a DFP pipeline that performs inference on Duo authentication logs. The goal is to use the pretrained models generated in the Duo Training notebook to generate anomaly scores for each log. These anomaly scores can be used by security teams to detect abnormal behavior when it happens so the proper action can be taken.

<div class="alert alert-block alert-info">
<b>Note:</b> For more information on DFP, the Morpheus pipeline, and setup steps to run this notebook, please see the coresponding DFP training materials.
</div>

In [1]:
# Ensure that the morpheus directory is in the python path. This may not need to be run depending on the environment setup
import sys
import os
sys.path.insert(0, os.path.abspath("./morpheus"))

In [2]:
import functools
import logging
import os
import typing
from datetime import datetime
from functools import partial

import click
import mlflow
from dfp.stages.dfp_file_batcher_stage import DFPFileBatcherStage
from dfp.stages.dfp_file_to_df import DFPFileToDataFrameStage
from dfp.stages.dfp_inference_stage import DFPInferenceStage
from dfp.stages.dfp_mlflow_model_writer import DFPMLFlowModelWriterStage
from dfp.stages.dfp_postprocessing_stage import DFPPostprocessingStage
from dfp.stages.dfp_preprocessing_stage import DFPPreprocessingStage
from dfp.stages.dfp_rolling_window_stage import DFPRollingWindowStage
from dfp.stages.dfp_split_users_stage import DFPSplitUsersStage
from dfp.stages.dfp_training import DFPTraining
from dfp.stages.multi_file_source import MultiFileSource
from dfp.utils.column_info import BoolColumn
from dfp.utils.column_info import ColumnInfo
from dfp.utils.column_info import CustomColumn
from dfp.utils.column_info import DataFrameInputSchema
from dfp.utils.column_info import DateTimeColumn
from dfp.utils.column_info import IncrementColumn
from dfp.utils.column_info import RenameColumn
from dfp.utils.column_info import StringCatColumn
from dfp.utils.column_info import create_increment_col
from dfp.utils.file_utils import date_extractor
from dfp.utils.file_utils import iso_date_regex

from morpheus._lib.file_types import FileTypes
from morpheus.cli.utils import get_package_relative_file
from morpheus.cli.utils import load_labels_file
from morpheus.config import Config
from morpheus.config import ConfigAutoEncoder
from morpheus.config import CppConfig
from morpheus.pipeline import LinearPipeline
from morpheus.stages.general.monitor_stage import MonitorStage
from morpheus.stages.output.write_to_file_stage import WriteToFileStage
from morpheus.utils.logger import configure_logging
from morpheus.utils.logger import get_log_levels
from morpheus.utils.logger import parse_log_level

# Left align all tables
from IPython.core.display import HTML
table_css = 'table {align:left;display:block}'
HTML('<style>{}</style>'.format(table_css))

## High Level Configuration

The following options significantly alter the functionality of the pipeline. These options are separated from the individual stage options since they may effect more than one stage. Additionally, the matching python script to this notebook, `dfp_duo_pipeline.py`, configures these options via command line arguments.

### Options

| Name | Type | Description |
| --- | --- | :-- |
| `train_users` | One of `["none"]` | For inference, this option should always be `"none"` |
| `skip_users` | List of strings | Any user in this list will be dropped from the pipeline. Useful for debugging to remove automated accounts with many logs. |
| `cache_dir` | string | The location to store cached files. To aid with development and reduce bandwidth, the Morpheus pipeline will cache data from several stages of the pipeline. This option configures the location for those caches. |
| `input_files` | List of strings | List of files to process. Can specificy multiple arguments for multiple files. Also accepts glob (\*) wildcards and schema prefixes such as `s3://`. For example, to make a local cache of an s3 bucket, use `filecache::s3://mybucket/*`. See `fsspec` documentation for list of possible options. |
| `model_name_formatter` | string | A format string to use when building the model name. The model name is constructed by calling `model_name_formatter.format(user_id=user_id)`. For example, with `model_name_formatter="my_model-{user_id}"` and a user ID of `"first:last"` would result in the model name of `"my_model-first:last"`. This should match the value used in `DFPMLFlowModelWriterStage`. Available keyword arguments: `user_id`, `user_md5`. |
| `experiment_name_formatter` | string | A format string (without the `f`) that will be used when creating an experiment in ML Flow. Available keyword arguments: `user_id`, `user_md5`, `reg_model_name`. |


In [3]:
# Global options
train_users = "none"

# Enter any users to skip here
skip_users: typing.List[str] = []

# Location where cache objects will be saved
cache_dir = "./.cache/dfp"

# Input files to read from
input_files = [
    "/work/examples/data/dfp/duo/duotest_pt1.json",
    "/work/examples/data/dfp/duo/duotest_pt2.json",
    "/work/examples/data/dfp/duo/duotest_pt3.json",
    "/work/examples/data/dfp/duo/duotest_pt4.json"
]

# The format to use for models
model_name_formatter = "DFP-duo-{user_id}"

# === Derived Options ===
# To include the generic, we must be training all or generic
include_generic = train_users == "all" or train_users == "generic"

# To include individual, we must be either training or inferring
include_individual = train_users != "generic"

# None indicates we arent training anything
is_training = train_users != "none"

### Global Config Object
Before creating the pipeline, we need to setup logging and set the parameters for the Morpheus config object. This config object is responsible for the following:
 - Indicating whether to use C++ or Python stages
    - C++ stages are not supported for the DFP pipeline. This should always be `False`
 - Setting the number of threads to use in the pipeline. Defaults to the thread count of the OS.
 - Sets the feature column names that will be used in model training
    - This option allows extra columns to be used in the pipeline that will not be part of the training algorithm.
    - The final features that the model will be trained on will be an intersection of this list with the log columns.
 - The column name that indicates the user's unique identifier
    - It is required for DFP to have a user ID column
 - The column name that indicates the timestamp for the log
    - It is required for DFP to know when each log occurred

In [4]:
# Enable the Morpheus logger
configure_logging(log_level=logging.DEBUG)

config = Config()

CppConfig.set_should_use_cpp(False)

config.num_threads = os.cpu_count()

config.ae = ConfigAutoEncoder()

config.ae.feature_columns = [
    'accessdevicebrowser', 'accessdeviceos', 'device', 'result', 'reason', 'logcount', "locincrement"
]
config.ae.userid_column_name = "username"
config.ae.timestamp_column_name = "timestamp"

In [6]:
# Specify the column names to ensure all data is uniform
source_column_info = [
    DateTimeColumn(name=config.ae.timestamp_column_name, dtype=datetime, input_name="timestamp"),
    RenameColumn(name=config.ae.userid_column_name, dtype=str, input_name="user.name"),
    RenameColumn(name="accessdevicebrowser", dtype=str, input_name="access_device.browser"),
    RenameColumn(name="accessdeviceos", dtype=str, input_name="access_device.os"),
    StringCatColumn(name="location",
                    dtype=str,
                    input_columns=[
                        "access_device.location.city",
                        "access_device.location.state",
                        "access_device.location.country"
                    ],
                    sep=", "),
    RenameColumn(name="authdevicename", dtype=str, input_name="auth_device.name"),
    BoolColumn(name="result",
                dtype=bool,
                input_name="result",
                true_values=["success", "SUCCESS"],
                false_values=["denied", "DENIED", "FRAUD"]),
    ColumnInfo(name="reason", dtype=str),
    # CustomColumn(name="user.groups", dtype=str, process_column_fn=partial(column_listjoin, col_name="user.groups"))
]

source_schema = DataFrameInputSchema(json_columns=["access_device", "application", "auth_device", "user"],
                                        column_info=source_column_info)


In [7]:
# Preprocessing schema
preprocess_column_info = [
    ColumnInfo(name=config.ae.timestamp_column_name, dtype=datetime),
    ColumnInfo(name=config.ae.userid_column_name, dtype=str),
    ColumnInfo(name="accessdevicebrowser", dtype=str),
    ColumnInfo(name="accessdeviceos", dtype=str),
    ColumnInfo(name="authdevicename", dtype=str),
    ColumnInfo(name="result", dtype=bool),
    ColumnInfo(name="reason", dtype=str),
    # Derived columns
    IncrementColumn(name="logcount",
                    dtype=int,
                    input_name=config.ae.timestamp_column_name,
                    groupby_column=config.ae.userid_column_name),
    CustomColumn(name="locincrement",
                    dtype=int,
                    process_column_fn=partial(create_increment_col, column_name="location")),
]

preprocess_schema = DataFrameInputSchema(column_info=preprocess_column_info, preserve_columns=["_batch_id"])


## Pipeline Construction
From this point on we begin constructing the stages that will make up the pipeline. To make testing easier, constructing the pipeline object, adding the stages, and running the pipeline, is provided as a single cell. The below cell can be rerun multiple times as needed for debugging.

### Source Stage (`MultiFileSource`)

This pipeline read input logs from one or more input files. This source stage will read all specified log files, combine them into a single `DataFrame`, and pass it into the pipeline. Once all of the logs have been read, the source completes. 

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `filenames` | List of strings | | Any files to read into the pipeline. All files will be combined into a single `DataFrame` |

### File Batcher Stage (`DFPFileBatcherStage`)

To improve performance, multiple small input files can be batched together into a single DataFrame for processing. This stage is responsible for determining the timestamp of input files, grouping input files into batches by time, and sending the batches to be processed into a single DataFrame. Batches of files that have been seen before will be loaded from cache resulting in increased performance. For example, when performaing a 60 day training run, 59 days can be cached with a period of `"D"` and retraining once per day.

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `period` | `str` | `"D"` | The period to create batches. See `pandas` windowing frequency documentation for available options.  |
| `date_conversion_func` | Function of `typing.Callable[[fsspec.core.OpenFile], datetime]` | | A callback which is responsible for determining the date for a specified file. |

### File to DataFrame Stage (`DFPFileToDataFrameStage`)

After files have been batched into groups, this stage is responsible for reading the files and converting into a DataFrame. The specified input schema converts the raw DataFrame into one suitable for caching and processing. Any columns that are not needed should be excluded from the schema.

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `schema` | `DataFrameInputSchema` | | After the raw `DataFrame` is read from each file, this schema will be applied to ensure a consisten output from the source. |
| `file_type` | `FileTypes` | `FileTypes.Auto` | Allows overriding the file type. When set to `Auto`, the file extension will be used. Options are `CSV`, `JSON`, `Auto`. |
| `parser_kwargs` | `dict` | `{}` | This dictionary will be passed to the `DataFrame` parser class. Allows for customization of log parsing. |
| `cache_dir` | `str` | `./.cache/dfp` | The location to write cached input files to. |

### Split Users Stage (`DFPSplitUsersStage`)

Once the input logs have been read into a `DataFrame`, this stage is responsible for breaking that single `DataFrame` with many users into multiple `DataFrame`s for each user. This is also where the pipeline chooses whether to train individual users or the generic user (or both).

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `include_generic` | `bool` | | Whether or not to combine all user logs into a single `DataFrame` with the username 'generic_user' |
| `include_individual` | `bool` | | Whether or not to output individual `DataFrame` objects for each user |
| `skip_users` | List of `str` | `[]` | Any users to remove from the `DataFrame`. Useful for debugging to remove automated accounts with many logs. Mutually exclusive with `only_users`. |
| `only_users` | List of `str` | `[]` | Only allow these users in the final `DataFrame`. Useful for debugging to focus on specific users. Mutually exclusive with `skip_users`. |

### Rolling Window Stage (`DFPRollingWindowStage`)

The Rolling Window Stage performs several key pieces of functionality for DFP.
1. This stage keeps a moving window of logs on a per user basis
   1. These logs are saved to disk to reduce memory requirements between logs from the same user
1. It only emits logs when the window history requirements are met
   1. Until all of the window history requirements are met, no messages will be sent to the rest of the pipeline.
   1. See the below options for configuring the window history requirements
1. It repeats the necessary logs to properly calculate log dependent features.
   1. To support all column feature types, incoming log messages can be combined with existing history and sent to downstream stages.
   1. For example, to calculate a feature that increments a counter for the number of logs a particular user has generated in a single day, we must have the user's log history for the past 24 hours. To support this, this stage will combine new logs with existing history into a single `DataFrame`.
   1. It is the responsibility of downstream stages to distinguish between new logs and existing history.

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `min_history` | `int` | `1` | The minimum number of logs a user must have before emitting any messages. Logs below this threshold will be saved to disk. |
| `min_increment` | `int` or `str` | `0` | Once the min history requirement is met, this stage must receive `min_increment` *new* logs before emmitting another message. Logs received before this threshold is met will be saved to disk. Can be specified as an integer count or a string duration. |
| `max_history` | `int` or `str` | `"1d"` | Once `min_history` and `min_increment` requirements have been met, this puts an upper bound on the maximum number of messages to forward into the pipeline and also the maximum amount of messages to retain in the history. Can be specified as an integer count or a string duration. |
| `cache_dir` | `str` | `./.cache/dfp` | The location to write log history to disk. |

### Preprocessing Stage (`DFPPreprocessingStage`)

This stage performs the final, row dependent, feature calculations as specified by the input schema object. Once calculated, this stage can forward on all received logs, or optionally can only forward on new logs, removing any history information.

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `input_schema` | `DataFrameInputSchema` | | The final, row dependent, schema to apply to the incoming columns |
| `only_new_batches` | `bool` | | Whether or not to foward on all received logs, or just new logs. |

### Inference Stage (`DFPInference`)

This stage performs several tasks to aid in performing inference. This stage will:
1. Download models as needed from MLFlow
1. Cache previously downloaded models to improve performance
   1. Models in the cache will be periodically refreshed from MLFlow at a configured rate
1. Perform inference using the downloaded model

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `model_name_formatter` | `str` | `""` | A format string to use when building the model name. The model name is constructed by calling `model_name_formatter.format(user_id=user_id)`. For example, with `model_name_formatter="my_model-{user_id}"` and a user ID of `"first:last"` would result in the model name of `"my_model-first:last"`. This should match the value used in `DFPMLFlowModelWriterStage` |

### Post Processing Stage (`DFPPostprocessingStage`)

This stage filters the output from the inference stage for any anomalous messages. Logs which exceed the specified Z-Score will be passed onto the next stage. All remaining logs which are below the threshold will be dropped.

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `z_score_threshold` | `float` | `2.0` | The Z-Score used to separate anomalous logs from normal logs. All normal logs will be filterd out and anomalous logs will be passed on. |

### Write to File Stage (`WriteToFileStage`)

This final stage will write all received messages to a single output file in either CSV or JSON format.

| Name | Type | Default | Description |
| --- | --- | --- | :-- |
| `filename` | `str` | | The file to write anomalous log messages to. |
| `overwrite` | `bool` | `False` | If the file specified in `filename` already exists, it will be overwritten if this option is set to `True` |

In [8]:
# Create a linear pipeline object
pipeline = LinearPipeline(config)

# Source stage
pipeline.set_source(MultiFileSource(config, filenames=input_files))

# Batch files into buckets by time. Use the default ISO date extractor from the filename
pipeline.add_stage(
    DFPFileBatcherStage(config,
                        period="D",
                        date_conversion_func=functools.partial(date_extractor, filename_regex=iso_date_regex)))

# Output is S3 Buckets. Convert to DataFrames. This caches downloaded S3 data
pipeline.add_stage(
    DFPFileToDataFrameStage(config,
                            schema=source_schema,
                            file_type=FileTypes.JSON,
                            parser_kwargs={
                                "lines": False, "orient": "records"
                            },
                            cache_dir=cache_dir))


# This will split users or just use one single user
pipeline.add_stage(
    DFPSplitUsersStage(config,
                        include_generic=include_generic,
                        include_individual=include_individual,
                        skip_users=skip_users))

# Next, have a stage that will create rolling windows
pipeline.add_stage(
    DFPRollingWindowStage(
        config,
        min_history=300 if is_training else 1,
        min_increment=300 if is_training else 0,
        # For inference, we only ever want 1 day max
        max_history="60d" if is_training else "1d",
        cache_dir=cache_dir))

# Output is UserMessageMeta -- Cached frame set
pipeline.add_stage(DFPPreprocessingStage(config, input_schema=preprocess_schema, only_new_batches=not is_training))

# Perform inference on the preprocessed data
pipeline.add_stage(DFPInferenceStage(config, model_name_formatter=model_name_formatter))

# Filter for only the anomalous logs
pipeline.add_stage(DFPPostprocessingStage(config, z_score_threshold=2.0))

# Write all anomalies to a CSV file
pipeline.add_stage(WriteToFileStage(config, filename="dfp_detections_duo.csv", overwrite=True))

# Run the pipeline
await pipeline._do_run()


====Registering Pipeline====
====Registering Pipeline Complete!====
====Starting Pipeline====
====Pipeline Started====
====Building Pipeline====
Added source: <from-multi-file-0; MultiFileSource(input_schema=DataFrameInputSchema(json_columns=['access_device', 'application', 'auth_device', 'user'], column_info=[RenameColumn(name='accessdevicebrowser', dtype=<class 'str'>, input_name='access_device.browser'), RenameColumn(name='accessdeviceos', dtype=<class 'str'>, input_name='access_device.os'), RenameColumn(name='locationcity', dtype=<class 'str'>, input_name='auth_device.location.city'), RenameColumn(name='device', dtype=<class 'str'>, input_name='auth_device.name'), BoolColumn(name='result', dtype=<class 'bool'>, input_name='result', value_map={'success': True, 'SUCCESS': True, 'denied': False, 'DENIED': False, 'FRAUD': False}), RenameColumn(name='reason', dtype=<class 'str'>, input_name='reason'), RenameColumn(name='username', dtype=<class 'str'>, input_name='user.name'), RenameColu

W20220827 06:58:36.361177   496 thread.cpp:138] unable to set memory policy - if using docker use: --cap-add=sys_nice to allow membind


Sending 4000 rows
Batch split users complete. Input: 4000 rows from 2021-09-14 00:52:14 to 2022-01-24 15:23:41. Output: 10 users, rows/user min: 176, max: 700, avg: 400.00. Duration: 5.55 ms
Rolling window complete for badguy in 16.03 ms. Input: 490 rows from 2021-09-14 00:52:14 to 2022-01-24 09:30:26. Output: 490 rows from 2021-09-14 00:52:14 to 2022-01-24 09:30:26
Preprocessed 490 data for logs in 2021-09-14 00:52:14 to 2022-01-24 09:30:26 in 34.24859046936035 ms
Rolling window complete for maliciousactor in 37.56 ms. Input: 700 rows from 2021-09-14 16:58:15 to 2022-01-24 15:23:41. Output: 700 rows from 2021-09-14 16:58:15 to 2022-01-24 15:23:41
Rolling window complete for usera in 24.63 ms. Input: 215 rows from 2021-10-07 10:53:23 to 2022-01-24 15:21:18. Output: 215 rows from 2021-10-07 10:53:23 to 2022-01-24 15:21:18
Rolling window complete for userb in 28.06 ms. Input: 457 rows from 2021-10-07 04:02:07 to 2022-01-24 15:22:10. Output: 457 rows from 2021-10-07 04:02:07 to 2022-01-24